<a href="https://colab.research.google.com/github/YASSINEABN/Zero_Chommage/blob/main/Copy_of_Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # Use this to upload your .pt files manually


In [ ]:
!git clone https://github.com/Rudrabha/Wav2Lip
%cd Wav2Lip

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 409 (delta 1), reused 0 (delta 0), pack-reused 405 (from 3)
Receiving objects: 100% (409/409), 549.58 KiB | 13.08 MiB/s, done.
Resolving deltas: 100% (226/226), done.
/content/Wav2Lip


In [1]:
#edit on inference.py



# from os import listdir, path
# import numpy as np
# if not hasattr(np, 'complex'):
#     np.complex = complex
#     np.float = float
# import scipy, cv2, os, sys, argparse, audio
# import json, subprocess, random, string
# from tqdm import tqdm
# from glob import glob
# import torch, face_detection
# from models import Wav2Lip
# import platform

# parser = argparse.ArgumentParser(description='Inference code to lip-sync videos in the wild using Wav2Lip models')

# parser.add_argument('--checkpoint_path', type=str, help='Name of saved checkpoint to load weights from', required=True)
# parser.add_argument('--face', type=str, help='Filepath of video/image that contains faces to use', required=True)
# parser.add_argument('--audio', type=str, help='Filepath of video/audio file to use as raw audio source', required=True)
# parser.add_argument('--outfile', type=str, help='Video path to save result. See default for an e.g.', default='results/result_voice.mp4')
# parser.add_argument('--static', type=bool, help='If True, then use only first video frame for inference', default=False)
# parser.add_argument('--fps', type=float, help='Can be specified only if input is a static image (default: 25)', default=25., required=False)
# parser.add_argument('--pads', nargs='+', type=int, default=[0, 10, 0, 0], help='Padding (top, bottom, left, right). Please adjust to include chin at least')
# parser.add_argument('--face_det_batch_size', type=int, help='Batch size for face detection', default=16)
# parser.add_argument('--wav2lip_batch_size', type=int, help='Batch size for Wav2Lip model(s)', default=128)
# parser.add_argument('--resize_factor', default=1, type=int, help='Reduce the resolution by this factor. Sometimes, best results are obtained at 480p or 720p')
# parser.add_argument('--crop', nargs='+', type=int, default=[0, -1, 0, -1], help='Crop video to a smaller region (top, bottom, left, right). Applied after resize_factor and rotate arg. Useful if multiple face present. -1 implies the value will be auto-inferred based on height, width')
# parser.add_argument('--box', nargs='+', type=int, default=[-1, -1, -1, -1], help='Specify a constant bounding box for the face. Use only as a last resort if the face is not detected. Also, might work only if the face is not moving around much. Syntax: (top, bottom, left, right).')
# parser.add_argument('--rotate', default=False, action='store_true', help='Sometimes videos taken from a phone can be flipped 90deg. If true, will flip video right by 90deg. Use if you get a flipped result, despite feeding a normal looking video')
# parser.add_argument('--nosmooth', default=False, action='store_true', help='Prevent smoothing face detections over a short temporal window')

# args = parser.parse_args()
# args.img_size = 96

# if os.path.isfile(args.face) and args.face.split('.')[1] in ['jpg', 'png', 'jpeg']:
# 	args.static = True

# def get_smoothened_boxes(boxes, T):
# 	for i in range(len(boxes)):
# 		if i + T > len(boxes):
# 			window = boxes[len(boxes) - T:]
# 		else:
# 			window = boxes[i : i + T]
# 		boxes[i] = np.mean(window, axis=0)
# 	return boxes

# def load_model(path):
#     print("Load checkpoint from: {}".format(path))
#     if device == 'cuda':
#         model = torch.jit.load(path).to(device)
#     else:
#         model = torch.jit.load(path, map_location='cpu')
#     model.eval()
#     return model

# def face_detect(images):
# 	detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D, flip_input=False, device=device)
# 	batch_size = args.face_det_batch_size
# 	while 1:
# 		predictions = []
# 		try:
# 			for i in tqdm(range(0, len(images), batch_size)):
# 				predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
# 		except RuntimeError:
# 			if batch_size == 1:
# 				raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
# 			batch_size //= 2
# 			print('Recovering from OOM error; New batch size: {}'.format(batch_size))
# 			continue
# 		break
# 	results = []
# 	pady1, pady2, padx1, padx2 = args.pads
# 	for rect, image in zip(predictions, images):
# 		if rect is None:
# 			cv2.imwrite('temp/faulty_frame.jpg', image)
# 			raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')
# 		y1 = max(0, rect[1] - pady1)
# 		y2 = min(image.shape[0], rect[3] + pady2)
# 		x1 = max(0, rect[0] - padx1)
# 		x2 = min(image.shape[1], rect[2] + padx2)
# 		results.append([x1, y1, x2, y2])
# 	boxes = np.array(results)
# 	if not args.nosmooth: boxes = get_smoothened_boxes(boxes, T=5)
# 	results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]
# 	del detector
# 	return results

# def datagen(frames, mels):
# 	img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []
# 	if args.box[0] == -1:
# 		if not args.static:
# 			face_det_results = face_detect(frames)
# 		else:
# 			face_det_results = face_detect([frames[0]])
# 	else:
# 		print('Using the specified bounding box instead of face detection...')
# 		y1, y2, x1, x2 = args.box
# 		face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]
# 	for i, m in enumerate(mels):
# 		idx = 0 if args.static else i%len(frames)
# 		frame_to_save = frames[idx].copy()
# 		face, coords = face_det_results[idx].copy()
# 		face = cv2.resize(face, (args.img_size, args.img_size))
# 		img_batch.append(face)
# 		mel_batch.append(m)
# 		frame_batch.append(frame_to_save)
# 		coords_batch.append(coords)
# 		if len(img_batch) >= args.wav2lip_batch_size:
# 			img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)
# 			img_masked = img_batch.copy()
# 			img_masked[:, args.img_size//2:] = 0
# 			img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
# 			mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])
# 			yield img_batch, mel_batch, frame_batch, coords_batch
# 			img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []
# 	if len(img_batch) > 0:
# 		img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)
# 		img_masked = img_batch.copy()
# 		img_masked[:, args.img_size//2:] = 0
# 		img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
# 		mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])
# 		yield img_batch, mel_batch, frame_batch, coords_batch

# mel_step_size = 16
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print('Using {} for inference.'.format(device))

# def _load(checkpoint_path):
#     if device == 'cuda':
#         checkpoint = torch.jit.load(checkpoint_path)
#     else:
#         try:
#             checkpoint = torch.jit.load(checkpoint_path, map_location='cpu')
#         except RuntimeError:
#             checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
#     return checkpoint

# def main():
# 	pass  # Entire main logic is commented

# if __name__ == '__main__':
# 	main()


In [ ]:
!pip install -r requirements.txt
!sudo apt-get install ffmpeg



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86)

In [ ]:
# ✅ Install necessary packages with compatible versions
!pip install numpy==1.21.6
!pip install librosa==0.8.1
!pip install opencv-python
!pip install opencv-contrib-python
!pip install scipy matplotlib tqdm numba requests dlib


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23

In [ ]:
# Reinstall only compatible versions
!pip install numpy==1.21.6 librosa==0.8.1
!pip install opencv-python opencv-contrib-python scipy matplotlib tqdm numba requests dlib
!sudo apt-get install ffmpeg


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23

In [ ]:
!pip install Wav2Lipy

  Using cached librosa-0.7.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.17.1.zip (6.5 MB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of wav2lipy to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from wav2lipy) (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6

In [ ]:
!pip install pathimport
!pip install face-detection

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 765.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.0 MB/s eta 0:00:00
  Created wheel for face-detection: filename=face_detection-0.2.2-py3-none-any.whl size=25557 sha256=ee6ccccef2002bfecbd3f711b1825bf0239f9fc4871b5f40e122190e5fdc6

In [ ]:
import torch

try:
    # Try loading as TorchScript model on CPU explicitly:
    model = torch.jit.load('../nogan.pt', map_location='cpu')
    print("TorchScript model loaded on CPU")
except RuntimeError as e:
    print(f"RuntimeError loading TorchScript model: {e}")
    # If failed, try fallback loading as checkpoint dict, disable weights_only:
    model = torch.load('../nogan.pt', map_location='cpu', weights_only=False)
    print("Loaded with torch.load as checkpoint")


TorchScript model loaded on CPU


In [ ]:
!python inference.py \
  --checkpoint_path ../nogan.pt \
  --face ../RH.png \
  --audio ../speech.wav \
  --outfile ../result_nogan.mp4 \
  --pads 0 20 0 0 \
  --nosmooth

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 1
(80, 398)
Length of mel chunks: 121
  0% 0/1 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]